In [1]:
import pandas as pd
import torch as th
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
from tab_transformer_pytorch import TabTransformer
from matplotlib import pyplot as plt
# %matplotlib inline
import time

In [2]:
# train_data = pd.read_csv('../data/cbtc/csv/train.csv')
train_data = pd.read_csv('../../data/cbtc/csv/new/1.hcz2cq/train.csv')
test_data = pd.read_csv('../../data/cbtc/csv/new/1.hcz2cq/test.csv')

# FEATURES = ['brake','target','speed','slope']
FEATURES = ['brake','target','speed','slope']
# LABEL = ['acc']
LABEL = ['acc']

train_features = train_data.loc[:, FEATURES]
train_label = train_data.loc[:, LABEL]

test_features = test_data.loc[:, FEATURES]
# test_labels = test_data.loc[1:, ['acc']]
test_label = test_data.loc[:, LABEL]

In [3]:
# # 输入标准化
# from sklearn.preprocessing import StandardScaler
# train_features = StandardScaler().fit_transform(train_features)
# test_features = StandardScaler().fit_transform(test_features)

In [4]:
# train_features

In [5]:
def data_handler(x_input):
    x_array = np.array(x_input)
    x_tensor = th.tensor(x_array)

    return x_tensor

In [6]:
train_features = data_handler(train_features)
train_label = data_handler(train_label)

test_features = data_handler(test_features)
test_label = data_handler(test_label)

In [7]:
batch_size = 32
epoch_nums = 160
learn_rate = 0.0001
wgt_dcy = 0.1
# 实际速度放大100倍, loss除以1e4
multiple = 100

In [8]:
class DatasetHandler(Dataset):
    def __init__(self,x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, idx):
        return self.x[idx,:], self.y[idx]

# train_dataset = DatasetHandler(train_features, train_label)
# train_dataset = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [9]:
train_dataset = DatasetHandler(train_features, train_label)
train_dataset = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = DatasetHandler(test_features, test_label)
test_dataset = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [10]:
model = TabTransformer(
    categories = (2500, 2500, 2500), # 离散
    num_continuous = 1, # 连续
    dim = 32, # dimension, paper set at 32
    dim_out = 1, # 输出维度
    depth = 6, # depth, paper recommended 6
    heads = 8, # heads, paper recommends 8
    attn_dropout = 0.1, # post-attention dropout
    ff_dropout = 0.1, # feed forward dropout
    mlp_hidden_mults = (4, 2), # relative multiples of each hidden dimension
    mlp_act = nn.ReLU(), # activation for final mlp, defaults to relu
)

In [11]:
cost = nn.MSELoss(reduction='mean')
optimizer = th.optim.Adam(model.parameters(), lr=learn_rate, weight_decay=wgt_dcy)

In [ ]:
# loss_all = 0
start_time = time.time()

for epoch in range(epoch_nums):
    loss_all = 0
    model.train()
    
    for i, data in enumerate(train_dataset, 0):
        features, label = data
            
        features_spt = th.split(features, 3, dim=1) # 拆分输入
        features_ctg = features_spt[0] # 离散变量
        features_ctn = features_spt[1].to(th.float32) # 坡度
            
        label = label.to(th.float32) # 加速度

        optimizer.zero_grad()
            
        y_pre = model(features_ctg, features_ctn)
        loss = cost(y_pre, label)
        loss_all += loss

#         optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    loss_all /= len(train_dataset)*multiple*multiple
    print("epoch: {}/{}, loss = {:.6f}".format(epoch + 1, epoch_nums, loss_all))
print(time.time()-start_time)

In [13]:
def valid(models, dataset):
    models.eval()
    total_loss = 0.0

    with th.no_grad():
        for i, data in enumerate(dataset, 0):
            # 得到数据和标签
            features, label = data

            feature_spt = th.split(features, 3, dim=1) # 拆分输入
            feature_ctg = feature_spt[0] # 离散变量
            feature_ctn = feature_spt[1].to(th.float32) # 坡度
            # feature_ctg, feature_ctn = split(features)
            label = label.to(th.float32)

            # 预测
            pre = models(feature_ctg, feature_ctn)
            total_loss += cost(pre, label).item()
            # print(i)

    total_loss /= len(dataset)*multiple*multiple

    print("valid loss = {:.6f}".format(total_loss))

In [ ]:
valid(model, test_dataset)

In [17]:
def predict(test_input):
    model.eval()
    feature_spt = th.split(test_input, 3, dim=1)
    feature_ctg = feature_spt[0]
    feature_ctn = feature_spt[1].to(th.float32)

    pred_data = model(feature_ctg, feature_ctn)
    return pred_data

In [18]:
test_pred = predict(test_features)
test_pred = test_pred.data.numpy()

In [ ]:
y_pred = test_pred # 预测加速度
y_test = test_label # 实际加速度

# 画出加速度-时间曲线
plt.figure(figsize=(16, 8))
plt.title("Train Acceleration")
plt.xlabel('Time / s')
plt.ylabel('Acceleration m/s')

data_len = len(y_test) - 1
x = 0.2 * np.linspace(0, data_len, data_len+1, endpoint=True)
plt.plot(x, y_test /100, 'b', linewidth=1, label="test acceleration")
plt.plot(x, y_pred /100, 'r', linewidth=1, label="pred acceleration")
plt.legend()

plt.show()

In [ ]:
# test = test_data.iloc[:, 2] # test speed
test = test_data.loc[:, ['speed']] # test speed
pred = np.zeros(len(test)) # pred speed

for idx in range(len(test)):
    pred[idx] = sum(test_pred[0:idx]) / 5

# 画出速度-时间曲线
plt.figure(figsize=(16, 8))
plt.title("Train Speed")
plt.xlabel('Time / s')
plt.ylabel('Speed m/s')

data_len = len(test) - 1
x = 0.2 * np.linspace(0, data_len, data_len+1, endpoint=True)

plt.plot(x, test / 100, 'b', linewidth=1, label="test speed")
plt.plot(x, pred / 100, 'r', linewidth=1, label="pred speed")
plt.legend()

plt.show()